In [2]:
import numpy as np
import math
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
pauli_i = [I, 0]
pauli_x = [X, 1]
pauli_y = [Y, 2]
pauli_z = [Z, 3]

def get_basis_set(num_of_qubit = 1):
    basis_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    int_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    temp_set = []
    for i in range (num_of_qubit-1):
        for j in range (len(basis_set)):
            for k in range (len(int_set)):
                a = basis_set[j]
                b = int_set[k]
                #temp_set.append()
                temp_set.append([np.kron(a[0],b[0]),[a[1],b[1]]])
        int_set = temp_set
        #print(int_set)
        temp_set = []
    return int_set

def get_decomposition(A, num_of_q):
    basis = get_basis_set(num_of_q)
    dec_list = []
    for i in range(len(basis)):
        b = basis[i]
        coeff = np.trace(A*b[0])/(2**num_of_q)
        dec_list.append([coeff, b[0], b[1]])
    return dec_list
def get_arbitary_decomposition(A):
    size = A.shape
    size = size[0]
    power = math.log2(size)
    p = int(power)
    if (p != power):
        p += 1
    ret = np.zeros([2**p,2**p], dtype = float)
    ret[:size, :size] = A
    for i in range(size, 2**p):
        ret[i,i] = 1
    return get_decomposition(ret, p)


In [4]:
import numpy as np
from scipy.optimize import minimize
import math
import time

start = time.clock()
a = np.matrix('11.5384615384615,-4.80769230769231,2.17948717948718,-0.961538461538462;-4.80769230769231,8.84615384615385,-0.961538461538462,1.69230769230769;-2.30769230769231,0.961538461538462,10.8974358974359,-4.80769230769231;0.961538461538462,-1.76923076923077,-4.80769230769231,8.46153846153846')
#a = np.matrix('1 2 3 4; 1 5 5 3; 2 3 4 5; 7 9 8 1')
dec_list = get_arbitary_decomposition(a)
decomp=[]
for i in range(len(dec_list)):
    if (abs(dec_list[i][0]) != 0):
        decomp.append(dec_list[i])
#print(len(decomp))
#a = a/np.linalg.norm(a)
#a = a/10
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
zKet = np.asmatrix('1;0') #np.asmatrix('1;0')
H = (1/math.sqrt(2))*np.asmatrix('1 1;1 -1')
U = np.asmatrix('-0.535897435897435, 0, 0, 0;-0.211538461538460, -0.211538461538460, 0, 0;0.0205128205128212, 0, 0.0205128205128212, 0;0.0923076923076916, 0, 0, 0.0923076923076916')
#U = U/np.linalg.norm(a)
#a = a/np.linalg.norm(a)
#U = U/100
a = a/100
#U = U/10
zKet = np.kron(zKet,zKet)
print(U*zKet)
lr = 0.001
#print(U)
#ans = np.linalg.inv(a)*U*zKet
#print(ans)
#print((ans)/np.linalg.norm(ans))
z = np.asmatrix('1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1')

#print(len(decomp))
#calculate cost
def calculate_cost(params):
    alpha = params[0]
    alpha2 = params[2]
    alpha3 = params[1]
    alpha4 = params[3]
    Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
    Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
    Ry = np.kron(Ry1,Ry2)
    Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
    Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
    Ry5 = np.kron(Ry3,Ry4)
    V = Ry*z*Ry5
    #V = Ry*Ry5
    cparam1, cparam2 = 0+0j, 0+0j
    for i in range(len(decomp)):
        for j in range(len(decomp)):
            al = decomp[i][1]
            ald = decomp[j][1]
            param1 = 0
            param1 = np.transpose(zKet)@np.transpose(U)@al@V@zKet
            param1 = param1*np.transpose(zKet)@np.transpose(V)@ald.H@U@zKet
            param1 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam1 += param1 
            param2 = 0
            param2 = np.transpose(zKet) * np.transpose(V) * ald.H * al * V * zKet
            param2 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam2 += param2 
    cost = 1 - (cparam1/cparam2)
    cost = np.asarray(cost)
    cost = cost[0][0]
    #print("cost = " + str(cost))
    return abs(cost)


'''alpha = np.pi/6;
alpha1 = np.pi/8;
cost = calculate_cost(alpha, alpha1)'''
maxiter = 2000000
iters = 0 
threshold = 0.00000000000047
'''while((abs(cost) > threshold) and (maxiter > iters)):
    #print('##################################ITERATION - '+ str(iters) +'  ###############################')
    alpha, alpha1 = optimize(alpha, alpha1)
    cost = calculate_cost(alpha, alpha1)
    #print('##################################  END   ###############################\n')
    iters +=1
print(cost)
print(alpha,alpha1)
print(iters)
alpha2 = alpha1'''
from scipy.optimize import minimize
init_params = [np.pi/8, np.pi/4, np.pi/2, np.pi/16]
#print(init_params)
#calculate_cost(init_params)

out = minimize(calculate_cost, x0=init_params, method="COBYLA", options= {'maxiter':20000, 'disp': True}, tol = threshold)
#print(out)
stop = time.clock()
alpha = out['x'][0]
alpha2 = out['x'][2]
alpha3 = out['x'][1]
alpha4 = out['x'][3]
Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
Ry = np.kron(Ry1,Ry2)
Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
Ry5 = np.kron(Ry3,Ry4)
V = Ry*z*Ry5
print (V*zKet*-1)
print(stop-start)



c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


[[-0.53589744]
 [-0.21153846]
 [ 0.02051282]
 [ 0.09230769]]
[[-0.74406453]
 [-0.66403927]
 [-0.07124922]
 [ 0.01853031]]
8.1101831


c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:97: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [16]:
import numpy as np
from scipy.optimize import minimize
import math

#a = a/np.linalg.norm(a)
#a = a/10
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
zKet = np.asmatrix('1;0') #np.asmatrix('1;0')
H = (1/math.sqrt(2))*np.asmatrix('1 1;1 -1')
U = np.asmatrix('-0.535897435897435, 0, 0, 0;-0.211538461538460, -0.211538461538460, 0, 0;0.0205128205128212, 0, 0.0205128205128212, 0;0.0923076923076916, 0, 0, 0.0923076923076916')
#U = U/np.linalg.norm(a)
#a = a/np.linalg.norm(a)
U = U/100
a = a/100
#U = U/10
zKet = np.kron(zKet,zKet)
print(U*zKet)
lr = 0.001
#print(U)
ans = np.linalg.inv(a)*U*zKet
print(ans)
print((ans)/np.linalg.norm(ans))
z = np.asmatrix('1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1')
#calculate cost
def calculate_cost(params):
    alpha = params[0]
    alpha2 = params[2]
    alpha3 = params[1]
    alpha4 = params[3]
    Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
    Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
    Ry = np.kron(Ry1,Ry2)
    Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
    Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
    Ry5 = np.kron(Ry3,Ry4)
    V = Ry*z*Ry5
    #V = Ry*Ry5
    cparam1, cparam2 = 0+0j, 0+0j
    for i in range(len(decomp)):
        for j in range(len(decomp)):
            al = decomp[i][1]
            ald = decomp[j][1]
            param1 = 0
            param1 = np.transpose(zKet)@np.transpose(U)@al@V@zKet
            param1 = param1*np.transpose(zKet)@np.transpose(V)@ald.H@U@zKet
            param1 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam1 += param1 
            param2 = 0
            param2 = np.transpose(zKet) * np.transpose(V) * ald.H * al * V * zKet
            param2 *= decomp[i][0] * np.conj(decomp[j][0])
            cparam2 += param2 
    cost = 1 - (cparam1/cparam2)
    cost = np.asarray(cost)
    cost = cost[0][0]
    #print("cost = " + str(cost))
    return abs(cost)


'''alpha = np.pi/6;
alpha1 = np.pi/8;
cost = calculate_cost(alpha, alpha1)'''
maxiter = 2000000
iters = 0 
threshold = 0.000000047
'''while((abs(cost) > threshold) and (maxiter > iters)):
    #print('##################################ITERATION - '+ str(iters) +'  ###############################')
    alpha, alpha1 = optimize(alpha, alpha1)
    cost = calculate_cost(alpha, alpha1)
    #print('##################################  END   ###############################\n')
    iters +=1
print(cost)
print(alpha,alpha1)
print(iters)
alpha2 = alpha1'''
from scipy.optimize import minimize
init_params = [np.pi/8, np.pi/4, np.pi/2, np.pi/16]
print(init_params)
calculate_cost(init_params)

out = minimize(calculate_cost, x0=init_params, method="COBYLA", options= {'maxiter':20000, 'disp': True}, tol = threshold)
print(out)
alpha = out['x'][0]
alpha2 = out['x'][2]
alpha3 = out['x'][1]
alpha4 = out['x'][3]
Ry1 = np.matrix([[np.cos(alpha/2.), -1*np.sin(alpha/2.)], [np.sin(alpha/2.), np.cos(alpha/2.)]])
Ry2 = np.matrix([[np.cos(alpha3/2.), -1*np.sin(alpha3/2.)], [np.sin(alpha3/2.), np.cos(alpha3/2.)]])
Ry = np.kron(Ry1,Ry2)
Ry3 = np.matrix([[np.cos(alpha2/2.), -1*np.sin(alpha2/2.)], [np.sin(alpha2/2.), np.cos(alpha2/2.)]])
Ry4 = np.matrix([[np.cos(alpha4/2.), -1*np.sin(alpha4/2.)], [np.sin(alpha4/2.), np.cos(alpha4/2.)]])
Ry5 = np.kron(Ry3,Ry4)
V = Ry*z*Ry5
print (V*zKet)



[[-0.00535897]
 [-0.00211538]
 [ 0.00020513]
 [ 0.00092308]]
[[  26.85185185]
 [ 262.78490028]
 [-329.97150997]
 [  95.97578348]]
[[ 0.06195069]
 [ 0.60627868]
 [-0.76128686]
 [ 0.22142852]]
[0.39269908169872414, 0.7853981633974483, 1.5707963267948966, 0.19634954084936207]


KeyboardInterrupt: 